In [1]:
import pandas as pd
import numpy as np

In [ ]:
attr_df = pd.read_csv("/kaggle/input/celeba-dataset/list_attr_celeba.csv")
attr_df = attr_df[["image_id", "Smiling"]]
attr_df["Smiling"] = attr_df["Smiling"].map({1: 1, -1: 0})

landmarks_df = pd.read_csv("/kaggle/input/celeba-dataset/list_landmarks_align_celeba.csv")
landmarks_df.reset_index(inplace=True)
landmarks_df.rename(columns={"index": "img_id"}, inplace=True)

df = pd.merge(attr_df, landmarks_df, on="image_id")

df["mouth_width"] = df["rightmouth_x"] - df["leftmouth_x"]
df["mouth_height"] = ((df["rightmouth_y"] + df["leftmouth_y"]) / 2) - df["nose_y"]
df["mouth_slope"] = (df["rightmouth_y"] - df["leftmouth_y"]) / (
        df["rightmouth_x"] - df["leftmouth_x"] + 1e-5
)

features = df[["mouth_width", "mouth_height", "mouth_slope"]]
labels = df["Smiling"]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2, stratify=labels, random_state=42)

df["Smiling"] = df["Smiling"].astype(str)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

image_dir = "/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba"

train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=image_dir,
    x_col="image_id",
    y_col="Smiling",
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="training"
)

val_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=image_dir,
    x_col="image_id",
    y_col="Smiling",
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = Dense(64, activation="relu")(x)
x = Dropout(0.2)(x)
predictions = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss="binary_crossentropy",
              metrics=["accuracy"])

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)

In [ ]:
base_model.trainable = True
model.compile(optimizer=Adam(1e-5), loss="binary_crossentropy", metrics=["accuracy"])

model.fit(train_generator, validation_data=val_generator, epochs=5)

In [ ]:
model.export("saved_model/smile_detector")

In [ ]:
# Here is import of data

# from google.colab import files
# import shutil
# 
# shutil.make_archive('savedModelSmilingDetector', 'zip', '/content/saved_model')
# files.download('savedModelSmilingDetector.zip')

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model("/content/keras_model/smile_detector.keras")

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

In [ ]:
img_array = preprocess_image("/content/images/1.jpg")
prediction = model.predict(img_array)[0][0]

label = "Smiling" if prediction > 0.8 else "Not Smiling"
confidence = round(prediction if prediction > 0.9 else 1 - prediction, 3)

print(f"{label} with confidence {confidence}")

# Development pipeline of CNN MobilenetV2 model
### The whole model built on 6 layers with base model layer from keras, regulirization, pooling, two dense layers with RELU and Sigmoid activators, and predictions layer with image tensor preprocess.

During the analysis and dataset annotations preprocessing I discovered that actually celebrities not really grumpy and always smiling at paparazzi and trying play with them. That is why our model incomplete at finding if person on image genuinely not smiling, and drastically good at differentiating smiling ones.

For example, we getting an image where person not smiling it gives high confidence of that person is smiling between 50% and 80 %, but if person is smiling on image we receive higher than 85% of confidence, accordingly to this context we can assume the problematic of dataset being too smily. 

Problems with training:
* High volume of dataset leads to prolonging the generator of training and validating batches. 
* Again high volume of dataset leads to extremely long time for training simple feature of smiling with:
    df["mouth_width"] = df["rightmouth_x"] - df["leftmouth_x"]
    df["mouth_height"] = ((df["rightmouth_y"] + df["leftmouth_y"]) / 2) - df["nose_y"]
    df["mouth_slope"] = (df["rightmouth_y"] - df["leftmouth_y"]) / (
        df["rightmouth_x"] - df["leftmouth_x"] + 1e-5
)
* Small preprocessing issues connected with str, int, or float casting

Advantages of dataset that helps a lot:
* Comprehensive annotations with everyting you need to understand the distance of facial features
* Comfort format of annotations gives better understanding of how the smiling and not smiling ones distributed among the dataset 
* Resized format 224x224 for mobile models like: Mobilenet, small CNN architectures and gradient boosting performance.

TO-DO:
* I want to try extend the dataset with more non-smiling records 
* Apply gradient boosting on mobilenet as feature extractor for CatBoost pipeline learning